In [1]:
!pip install transformers lime hf_xet datasets captum numpy

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader
#from transformers import AutoTokenizer, AutoModelForCausalLM
from lime.lime_text import LimeTextExplainer
from datasets import load_dataset
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
import warnings
import tqdm as tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig
from captum.attr import IntegratedGradients
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
warnings.filterwarnings("ignore")


In [3]:
dataset = load_dataset("hatexplain")

README.md:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

hatexplain.py:   0%|          | 0.00/4.78k [00:00<?, ?B/s]

The repository for hatexplain contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hatexplain.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/15383 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1922 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1924 [00:00<?, ? examples/s]

In [4]:
def join_tokens(example):
    example["text"] = " ".join(example["post_tokens"])
    return example

dataset = dataset.map(join_tokens)

dataset["train"][0]["text"]

Map:   0%|          | 0/15383 [00:00<?, ? examples/s]

Map:   0%|          | 0/1922 [00:00<?, ? examples/s]

Map:   0%|          | 0/1924 [00:00<?, ? examples/s]

'u really think i would not have been raped by feral hindu or muslim back in india or bangladesh and a neo nazi would rape me as well just to see me cry'

In [5]:
def combine_rationale_masks(rationale_masks, method='weighted'):
    """
    Combines multiple binary rationale masks into a single mask.
    """
    rationale_array = np.array(rationale_masks)

    # Union method: If any annotator highlights a token, it is part of the rationale
    if method == 'union':
        final_rationale_mask = (rationale_array.sum(axis=0) > 0).astype(int)

    # Majority method: A token is part of the rationale if the majority of annotators highlight it
    elif method == 'majority':
        final_rationale_mask = (rationale_array.sum(axis=0) >= (len(rationale_masks) / 2)).astype(int)

    # Weighted method: Tokens are weighted based on how many annotators highlight it
    elif method == 'weighted':
        # Compute the average weight for each token (0-1 scale)
        weights = rationale_array.mean(axis=0)
        # Map average values to 0 or 1, with values < 1 being fractional
        final_rationale_mask = weights.tolist()

    else:
        raise ValueError("Unknown method. Choose from 'union', 'majority', or 'weighted'.")

    return final_rationale_mask

def safe_add_final_rationale(example):
    rationales = example.get('rationales', None)

    if not rationales or not isinstance(rationales, list) or not all(isinstance(r, list) for r in rationales):
        # Default to a zero mask if invalid or missing
        num_tokens = len(example.get('post_tokens', []))
        example['final_rationale_mask'] = [0] * num_tokens
        return example

    try:
        example['final_rationale_mask'] = combine_rationale_masks(rationales, method='union')
    except Exception as e:
        # Handle edge cases or shape mismatch
        num_tokens = len(example.get('post_tokens', []))
        example['final_rationale_mask'] = [0] * num_tokens

    return example

dataset = dataset.map(safe_add_final_rationale)

Map:   0%|          | 0/15383 [00:00<?, ? examples/s]

Map:   0%|          | 0/1922 [00:00<?, ? examples/s]

Map:   0%|          | 0/1924 [00:00<?, ? examples/s]

In [6]:
from collections import Counter

def resolve_hate_label(label_list):
    """
    Convert multi-class label (0: hate, 1: neither, 2: offensive) to binary label.
    Returns 'hate' if majority is 0 or 2; otherwise 'non-hate'.
    """
    if not label_list or len(label_list) != 3:
        return "non-detrimental"  # fallback
    counts = Counter(label_list)
    majority_label = counts.most_common(1)[0][0]
    return "detrimenal_content" if majority_label in [0, 2] else "non-detrimental"
def add_binary_label(example):
    example['binary_label'] = resolve_hate_label(example['annotators']['label'])
    return example

dataset = dataset.map(add_binary_label)

Map:   0%|          | 0/15383 [00:00<?, ? examples/s]

Map:   0%|          | 0/1922 [00:00<?, ? examples/s]

Map:   0%|          | 0/1924 [00:00<?, ? examples/s]

In [7]:
from torch.utils.data import Dataset

class HateXplainDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=128):
        """
        dataset: HuggingFace-style Dataset object (already mapped with final_rationale_mask)
        tokenizer: Tokenizer (e.g., from transformers) to tokenize 'text'
        max_length: Max token length for model inputs
        """
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_map = {"non_detrimenal": 0, "detrimental": 1}  # adjust if needed

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]

        # Tokenize text
        encoding = self.tokenizer(
            item['text'],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Convert label to binary
        label = 1 if item['binary_label'] == 'detrimenal_content' else 0

        # Final rationale mask (assumes already aligned with post_tokens)
        rationale = item.get('final_rationale_mask', [])

        # Pad rationale to max_length
        rationale_tensor = torch.zeros(self.max_length)
        rationale_values = rationale[:self.max_length]
        rationale_tensor[:len(rationale_values)] = torch.tensor(rationale_values, dtype=torch.float32)

        return {
            'input_ids': encoding['input_ids'].squeeze(0),  # [max_length]
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.float32),
            'rationale_mask': rationale_tensor
        }

In [8]:
import torch
import torch.nn as nn
from transformers import RobertaModel, RobertaConfig

class CustomRobertaClassifier(nn.Module):
    def __init__(self, pretrained_model_name, hidden_size=128, dropout_rate=0.3):
        super(CustomRobertaClassifier, self).__init__()

        self.roberta = RobertaModel.from_pretrained(pretrained_model_name)
        self.dropout = nn.Dropout(dropout_rate)

        # Intermediate dense layer
        self.dense = nn.Linear(self.roberta.config.hidden_size, hidden_size)
        self.relu = nn.ReLU()

        # Final classification layer (outputting 1 value for binary classification)
        self.classifier = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [batch_size, hidden_size]

        x = self.dropout(pooled_output)
        x = self.relu(self.dense(x))
        x = self.dropout(x)
        logits = self.classifier(x)

        return self.sigmoid(logits)





In [9]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


model = CustomRobertaClassifier(pretrained_model_name='roberta-base')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
train_dataset = HateXplainDataset(dataset['train'], tokenizer)
validation_dataset = HateXplainDataset(dataset['validation'], tokenizer)
test_dataset = HateXplainDataset(dataset['test'], tokenizer)

In [11]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

In [12]:
from sklearn.metrics import classification_report
def evaluate(model, dataloader, device='cuda'):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            preds = (outputs.squeeze() > 0.5).int()

            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().int().tolist())

    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f"Validation Accuracy: {acc:.4f}, F1 Score: {f1:.4f}")
    return f1

In [15]:
import torch
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model.to(device)

CustomRobertaClassifier(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Lay

In [33]:
import torch
model.load_state_dict(torch.load('/content/hatexplain_roberta_weights_noIG_0.8201.pth', map_location=torch.device('cuda')))

<All keys matched successfully>

In [34]:
evaluate(model, test_loader, device='cuda')

Evaluating: 100%|██████████| 241/241 [00:15<00:00, 15.38it/s]

Validation Accuracy: 0.7791, F1 Score: 0.8260


0.8260335652885796

In [35]:
import torch
model.load_state_dict(torch.load('/content/hatexplain_roberta_weights_5_IG_0.8335.pth', map_location=torch.device('cuda')))

<All keys matched successfully>

In [36]:
evaluate(model, test_loader, device='cuda')

Evaluating: 100%|██████████| 241/241 [00:15<00:00, 15.24it/s]

Validation Accuracy: 0.7879, F1 Score: 0.8268


0.8268251273344652

In [37]:
import torch
model.load_state_dict(torch.load('/content/hatexplain_roberta_weights_5_GXI_0.8212.pth', map_location=torch.device('cuda')))

<All keys matched successfully>

In [38]:
evaluate(model, test_loader, device='cuda')

Evaluating: 100%|██████████| 241/241 [00:13<00:00, 17.28it/s]

Validation Accuracy: 0.7801, F1 Score: 0.8251


0.8251343530384456

In [39]:
import torch
model.load_state_dict(torch.load('/content/hatexplain_roberta_weights_5_ATTN_0.8331.pth', map_location=torch.device('cuda')))

<All keys matched successfully>

In [40]:
evaluate(model, test_loader, device='cuda')

Evaluating: 100%|██████████| 241/241 [00:13<00:00, 17.84it/s]

Validation Accuracy: 0.7931, F1 Score: 0.8357


0.8356729975227085

In [41]:
import torch
model.load_state_dict(torch.load('/content/hatexplain_roberta_weights_LIME_0.8259.pth', map_location=torch.device('cuda')))

<All keys matched successfully>

In [42]:
evaluate(model, test_loader, device='cuda')

Evaluating: 100%|██████████| 241/241 [00:13<00:00, 17.99it/s]

Validation Accuracy: 0.7963, F1 Score: 0.8367


0.8366666666666667

In [13]:
from captum.attr import LayerIntegratedGradients
from lime.lime_text import LimeTextExplainer
from sklearn.preprocessing import LabelEncoder
import numpy as np
def calculate_iou(pred_mask, true_mask):
    """
    Calculates the Intersection over Union (IoU) between predicted mask and ground-truth mask.
    Both masks should be binary tensors (1 = important, 0 = not important).

    IoU = (Intersection) / (Union)
    """
    pred_mask = pred_mask.bool()
    true_mask = true_mask.bool()
    intersection = (pred_mask & true_mask).sum().float()
    union = (pred_mask | true_mask).sum().float()

    if union == 0:
        return torch.tensor(1.0) if intersection == 0 else torch.tensor(0.0)

    return intersection / union



def calculate_precision_recall(pred_mask, true_mask):
    """
    Calculates Precision and Recall between predicted and ground-truth masks.

    Precision = (Correctly Predicted Important Tokens) / (Predicted Important Tokens)
    Recall = (Correctly Predicted Important Tokens) / (Ground Truth Important Tokens)
    """
    pred_mask = pred_mask.bool()
    true_mask = true_mask.bool()
    true_positives = (pred_mask & true_mask).sum().float()

    precision = true_positives / (pred_mask.sum().float() + 1e-8)
    recall = true_positives / (true_mask.sum().float() + 1e-8)

    return precision, recall


def calculate_faithfulness_simple(model, tokenizer, input_ids, attention_mask, explanation_mask, target_label, device='cuda'):
    """
    Calculates a simple 'deletion faithfulness' score.

    Steps:
    1. Get model's original probability for the true class.
    2. Remove the important tokens (set them to [PAD] token).
    3. Recompute the model's probability for the same class.
    4. Faithfulness = (original probability) - (new probability)

    A higher drop means that the important tokens were truly important!
    """

    with torch.no_grad():
        # --- Original prediction ---
        original_prob = model(input_ids, attention_mask).squeeze()

        # --- Mask important tokens ---
        input_ids_masked = input_ids.clone()
        important_indices = explanation_mask.nonzero(as_tuple=True)[0]
        input_ids_masked[0, important_indices] = tokenizer.pad_token_id

        # --- New prediction after masking ---
        new_prob = model(input_ids_masked, attention_mask).squeeze()

    return original_prob - new_prob
def forward_func(single_id, single_mask):
    model.eval()

    input_id = single_id.long()
    attention = single_mask.long()

    out=model(input_id, attention)
    return out

def predict_proba(texts):
        inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            probs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask']).squeeze()
            if probs.dim() == 0:
                probs = probs.unsqueeze(0)
            probs = torch.stack([1 - probs, probs], dim=1)  # shape: [batch_size, 2]
        return probs.cpu().numpy()
def evaluate_IG(model, dataloader, device='cuda', method='topk', k=5, threshold=0.01):
    model.eval()
    ig = LayerIntegratedGradients(forward_func, model.roberta.embeddings)

    iou_scores = []
    precisions = []
    recalls = []
    faithfulness_scores = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating XAI Metrics"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            rationales = batch['rationale_mask'].to(device)

            outputs = model(input_ids, attention_mask)
            preds = (outputs.squeeze() > 0.5).int()

            for i in range(input_ids.size(0)):  # Iterate over batch
              pred = preds[i].item()
              label = labels[i].item()

              if not (pred == 1 and label == 1):
                continue

              input_id = input_ids[i].unsqueeze(0)
              attention = attention_mask[i].unsqueeze(0)
              rationale = rationales[i]

              attributions = ig.attribute(input_id, additional_forward_args = attention, target=0, n_steps=15)


              attributions = attributions.sum(dim=-1).squeeze(0)  # (seq_len,)
              norm = attributions.abs().sum()
              if norm > 0:
                  attributions = attributions / norm
               # --- Explanation Mask ---
              if method == 'threshold':
                  explanation_mask = (attributions.abs() > threshold).long()
              elif method == 'topk':
                  topk = torch.zeros_like(attributions)
                  if k > 0:
                      topk_indices = torch.topk(attributions.abs(), k).indices
                      topk[topk_indices] = 1
                  explanation_mask = topk
              else:
                  raise ValueError(f"Unknown method {method}")

              # --- Ground Truth Mask ---
              true_mask = rationale[:attributions.size(0)]

              iou = calculate_iou(explanation_mask, true_mask)
              precision, recall = calculate_precision_recall(explanation_mask, true_mask)
              faithfulness = calculate_faithfulness_simple(model, tokenizer, input_id, attention, explanation_mask, label, device)

              iou_scores.append(iou.item())
              precisions.append(precision.item())
              recalls.append(recall.item())
              faithfulness_scores.append(faithfulness.item())


    if len(iou_scores) == 0:
        print("No samples met the condition (pred==1 and label==1).")
        return None

    avg_iou = sum(iou_scores) / len(iou_scores)
    avg_precision = sum(precisions) / len(precisions)
    avg_recall = sum(recalls) / len(recalls)
    avg_f1 = 2 * avg_precision * avg_recall / (avg_precision + avg_recall + 1e-8)
    avg_faithfulness = sum(faithfulness_scores) / len(faithfulness_scores)

    print(f"Validation XAI Metrics -- IoU: {avg_iou:.4f}, Precision: {avg_precision:.4f}, Recall: {avg_recall:.4f}, F1: {avg_f1:.4f}, Faithfulness Drop: {avg_faithfulness:.4f}")

    return {
        'IoU': avg_iou,
        'Precision': avg_precision,
        'Recall': avg_recall,
        'F1': avg_f1,
        'Faithfulness': avg_faithfulness
    }

def evaluate_GXI(model, dataloader, tokenizer, device='cuda', method='topk', k=5, threshold=0.01):
    model.eval()
    embedding_layer = model.roberta.embeddings.word_embeddings

    iou_scores = []
    precisions = []
    recalls = []
    faithfulness_scores = []

    for batch in tqdm(dataloader, desc="Evaluating XAI Metrics"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        rationales = batch['rationale_mask'].to(device)

        outputs = model(input_ids, attention_mask)
        preds = (outputs.squeeze() > 0.5).int()

        for i in range(input_ids.size(0)):  # Iterate over each example in batch
            pred = preds[i].item()
            label = labels[i].item()

            if not (pred == 1 and label == 1):
                continue  # Only evaluate when prediction and label are both positive

            input_id = input_ids[i].unsqueeze(0)
            attention = attention_mask[i].unsqueeze(0)
            rationale = rationales[i]

            # Get embeddings and enable gradients
            embed_input = embedding_layer(input_id)
            embed_input.requires_grad_()
            embed_input.retain_grad()

            # Forward pass with embeddings
            model.zero_grad()
            output = model(inputs_embeds=embed_input, attention_mask=attention)
            target_logit = output.squeeze()

            # Backward pass to get gradients
            target_logit.sum().backward(retain_graph=True)
            grads = embed_input.grad

            # Grad x Input
            grad_x_input = (grads * embed_input).sum(dim=-1).squeeze(0).clamp(min=0)
            norm = grad_x_input.sum()
            if norm > 0:
                grad_x_input = grad_x_input / norm
            # --- Generate Explanation Mask ---
            if method == 'threshold':
                explanation_mask = (grad_x_input > threshold).long()
            elif method == 'topk':
                topk = torch.zeros_like(grad_x_input)
                if k > 0:
                    topk_indices = torch.topk(grad_x_input, k).indices
                    topk[topk_indices] = 1
                explanation_mask = topk.long()
            else:
                raise ValueError(f"Unknown method: {method}")

            # Ground truth rationale (same length)
            true_mask = rationale[:grad_x_input.size(0)]

            # Metrics
            iou = calculate_iou(explanation_mask, true_mask)
            precision, recall = calculate_precision_recall(explanation_mask, true_mask)
            faithfulness = calculate_faithfulness_simple(model, tokenizer, input_id, attention, explanation_mask, label, device)

            iou_scores.append(iou.item())
            precisions.append(precision.item())
            recalls.append(recall.item())
            faithfulness_scores.append(faithfulness.item())

    if len(iou_scores) == 0:
        print("No samples met the condition (pred==1 and label==1).")
        return None

    # Aggregate metrics
    avg_iou = sum(iou_scores) / len(iou_scores)
    avg_precision = sum(precisions) / len(precisions)
    avg_recall = sum(recalls) / len(recalls)
    avg_f1 = 2 * avg_precision * avg_recall / (avg_precision + avg_recall + 1e-8)
    avg_faithfulness = sum(faithfulness_scores) / len(faithfulness_scores)

    print(f"\nXAI Metrics:")
    print(f"IoU: {avg_iou:.4f}")
    print(f"Precision: {avg_precision:.4f}")
    print(f"Recall: {avg_recall:.4f}")
    print(f"F1: {avg_f1:.4f}")
    print(f"Faithfulness Drop: {avg_faithfulness:.4f}")

    return {
        'IoU': avg_iou,
        'Precision': avg_precision,
        'Recall': avg_recall,
        'F1': avg_f1,
        'Faithfulness': avg_faithfulness
    }


def evaluate_attention_explanations(model, dataloader, tokenizer, device='cuda', method='topk', k=5, threshold=0.01, attention_layer=-1):
    model.eval()

    iou_scores = []
    precisions = []
    recalls = []
    faithfulness_scores = []

    for batch in tqdm(dataloader, desc="Evaluating Attention Explanations"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        rationales = batch['rationale_mask'].to(device)

        outputs, attentions = model(input_ids=input_ids, attention_mask=attention_mask, output_attentions=True)
        preds = (outputs.squeeze() > 0.5).int()

        for i in range(input_ids.size(0)):
            pred = preds[i].item()
            label = labels[i].item()

            if not (pred == 1 and label == 1):
                continue

            input_id = input_ids[i]
            attention = attention_mask[i]
            rationale = rationales[i]

            # Get attentions for the selected layer: shape [num_heads, seq_len, seq_len]
            attn = attentions[attention_layer][i]  # shape: [num_heads, seq_len, seq_len]

            # Aggregate attention over all heads and take attention on [CLS] token (token 0)
            # Note: RoBERTa may use <s> instead of [CLS], but position 0 is still the first token.
            avg_attention = attn.mean(dim=0)[0]  # shape: [seq_len], attention from first token
            norm = avg_attention.sum()
            if norm > 0:
                avg_attention = avg_attention / norm
            # --- Generate Explanation Mask ---
            if method == 'threshold':
                explanation_mask = (avg_attention > threshold).long()
            elif method == 'topk':
                topk_mask = torch.zeros_like(avg_attention)
                if k > 0:
                    topk_indices = torch.topk(avg_attention, k).indices
                    topk_mask[topk_indices] = 1
                explanation_mask = topk_mask.long()
            else:
                raise ValueError(f"Unknown method: {method}")

            true_mask = rationale[:explanation_mask.size(0)]

            iou = calculate_iou(explanation_mask, true_mask)
            precision, recall = calculate_precision_recall(explanation_mask, true_mask)
            faithfulness = calculate_faithfulness_simple(model, tokenizer, input_id.unsqueeze(0), attention.unsqueeze(0), explanation_mask, label, device)

            iou_scores.append(iou.item())
            precisions.append(precision.item())
            recalls.append(recall.item())
            faithfulness_scores.append(faithfulness.item())

    if len(iou_scores) == 0:
        print("No samples met the condition (pred==1 and label==1).")
        return None

    # Aggregate metrics
    avg_iou = sum(iou_scores) / len(iou_scores)
    avg_precision = sum(precisions) / len(precisions)
    avg_recall = sum(recalls) / len(recalls)
    avg_f1 = 2 * avg_precision * avg_recall / (avg_precision + avg_recall + 1e-8)
    avg_faithfulness = sum(faithfulness_scores) / len(faithfulness_scores)

    print(f"\nAttention Explanation Metrics:")
    print(f"IoU: {avg_iou:.4f}")
    print(f"Precision: {avg_precision:.4f}")
    print(f"Recall: {avg_recall:.4f}")
    print(f"F1: {avg_f1:.4f}")
    print(f"Faithfulness Drop: {avg_faithfulness:.4f}")

    return {
        'IoU': avg_iou,
        'Precision': avg_precision,
        'Recall': avg_recall,
        'F1': avg_f1,
        'Faithfulness': avg_faithfulness
    }



def evaluate_lime_explanations(
    model,
    dataloader,
    tokenizer,
    device='cuda',
    num_features=5,
    threshold=0.2,
    mode='num_features'  # 'threshold' or 'num_features'
):
    model.eval()
    explainer = LimeTextExplainer(class_names=["non_detrimental", "detrimental"])

    iou_scores = []
    precisions = []
    recalls = []
    faithfulness_scores = []

    for batch in tqdm(dataloader, desc="Evaluating LIME Explanations"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        rationales = batch['rationale_mask'].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask).squeeze()
            preds = (outputs > 0.5).int()

        for i in range(input_ids.size(0)):
            pred = preds[i].item()
            label = labels[i].item()

            if not (pred == 1 and label == 1):
                continue

            input_id = input_ids[i]
            rationale = rationales[i]
            text = tokenizer.decode(input_id, skip_special_tokens=True)

            if mode == 'num_features':
                explanation = explainer.explain_instance(
                    text,
                    predict_proba,
                    num_features=num_features,
                    labels=[1],
                    num_samples=256
                )
                exp_words = [word for word, _ in explanation.as_list(label=1)]

            elif mode == 'threshold':
                explanation = explainer.explain_instance(
                    text,
                    predict_proba,
                    num_features=100,  # get more features for thresholding
                    labels=[1],
                    num_samples=256
                )
                exp_list = explanation.as_list(label=1)
                total_weight = sum(abs(weight) for _, weight in exp_list)
                exp_words = [word for word, weight in exp_list if abs(weight) / (total_weight + 1e-8) >= threshold]

            else:
                raise ValueError("mode must be either 'num_features' or 'threshold'")

            exp_ids = []
            for word in exp_words:
                word_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(word))
                exp_ids.extend(word_ids)

            explanation_mask = torch.zeros_like(input_id)
            for idx, token_id in enumerate(input_id):
                if token_id.item() in exp_ids:
                    explanation_mask[idx] = 1

            true_mask = rationale[:len(explanation_mask)]

            iou = calculate_iou(explanation_mask, true_mask)
            precision, recall = calculate_precision_recall(explanation_mask, true_mask)
            faithfulness = calculate_faithfulness_simple(
                model,
                tokenizer,
                input_id.unsqueeze(0),
                attention_mask[i].unsqueeze(0),
                explanation_mask,
                label,
                device
            )

            iou_scores.append(iou.item())
            precisions.append(precision.item())
            recalls.append(recall.item())
            faithfulness_scores.append(faithfulness.item())

    if len(iou_scores) == 0:
        print("No valid samples found.")
        return None

    avg_iou = sum(iou_scores) / len(iou_scores)
    avg_precision = sum(precisions) / len(precisions)
    avg_recall = sum(recalls) / len(recalls)
    avg_f1 = 2 * avg_precision * avg_recall / (avg_precision + avg_recall + 1e-8)
    avg_faithfulness = sum(faithfulness_scores) / len(faithfulness_scores)

    print(f"\nLIME Explanation Metrics:")
    print(f"IoU: {avg_iou:.4f}")
    print(f"Precision: {avg_precision:.4f}")
    print(f"Recall: {avg_recall:.4f}")
    print(f"F1: {avg_f1:.4f}")
    print(f"Faithfulness Drop: {avg_faithfulness:.4f}")

    return {
        'IoU': avg_iou,
        'Precision': avg_precision,
        'Recall': avg_recall,
        'F1': avg_f1,
        'Faithfulness': avg_faithfulness
    }




In [44]:
import torch

ks = [3,5,10,15,20,25]
thresholds = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
for k in ks:
  print(" ")
  print('Top K: ,' + str(k))

  model.load_state_dict(torch.load('/content/hatexplain_roberta_weights_5_IG_0.8335.pth', map_location=torch.device('cuda')))
  evaluate_IG(model, test_loader, device='cuda', method='topk', k=k, threshold=0.01)

for threshold in thresholds:
  print(" ")
  print('Threshold: ,' + str(threshold))
  model.load_state_dict(torch.load('/content/hatexplain_roberta_weights_5_IG_0.8335.pth', map_location=torch.device('cuda')))
  evaluate_IG(model, test_loader, device='cuda', method='threshold', k=5, threshold=threshold)



 
Top K: ,3


Evaluating XAI Metrics: 100%|██████████| 241/241 [04:08<00:00,  1.03s/it]


Validation XAI Metrics -- IoU: 0.0965, Precision: 0.3077, Recall: 0.1407, F1: 0.1931, Faithfulness Drop: 0.3909
 
Top K: ,5


Evaluating XAI Metrics: 100%|██████████| 241/241 [04:08<00:00,  1.03s/it]


Validation XAI Metrics -- IoU: 0.1300, Precision: 0.3010, Recall: 0.2395, F1: 0.2667, Faithfulness Drop: 0.4492
 
Top K: ,10


...

Validation XAI Metrics -- IoU: 0.1247, Precision: 0.2980, Recall: 0.2255, F1: 0.2567, Faithfulness Drop: 0.4491
 
Threshold: ,0.1


Evaluating XAI Metrics: 100%|██████████| 241/241 [04:07<00:00,  1.03s/it]


Validation XAI Metrics -- IoU: 0.0768, Precision: 0.2975, Recall: 0.1062, F1: 0.1565, Faithfulness Drop: 0.3512
 
Threshold: ,0.5


Evaluating XAI Metrics: 100%|██████████| 241/241 [04:06<00:00,  1.02s/it]

Validation XAI Metrics -- IoU: 0.0052, Precision: 0.0133, Recall: 0.0052, F1: 0.0075, Faithfulness Drop: 0.0467


In [45]:
import torch
import torch.nn as nn
from transformers import RobertaModel, RobertaConfig

class CustomRobertaClassifier(nn.Module):
    def __init__(self, pretrained_model_name, hidden_size=128, dropout_rate=0.3):
        super(CustomRobertaClassifier, self).__init__()

        self.roberta = RobertaModel.from_pretrained(pretrained_model_name)
        self.dropout = nn.Dropout(dropout_rate)

        # Intermediate dense layer
        self.dense = nn.Linear(self.roberta.config.hidden_size, hidden_size)
        self.relu = nn.ReLU()

        # Final classification layer (outputting 1 value for binary classification)
        self.classifier = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids = None, attention_mask = None, inputs_embeds = None):
      if inputs_embeds is not None:
          outputs = self.roberta(inputs_embeds= inputs_embeds, attention_mask=attention_mask)
      else:
          outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)

      pooled_output = outputs.pooler_output  # [batch_size, hidden_size]

      x = self.dropout(pooled_output)
      x = self.relu(self.dense(x))
      x = self.dropout(x)
      logits = self.classifier(x)

      return self.sigmoid(logits)


tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


model = CustomRobertaClassifier(pretrained_model_name='roberta-base')
model.to(device)



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CustomRobertaClassifier(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Lay

In [46]:
import torch

ks = [3,5,10,15,20,25]
thresholds = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
for k in ks:
  print(" ")
  print('Top K: ,' + str(k))
  model.load_state_dict(torch.load('/content/hatexplain_roberta_weights_5_GXI_0.8212.pth'))
  evaluate_GXI(model, test_loader, tokenizer,device='cuda', method='topk', k=k, threshold=0.01)

for threshold in thresholds:
  print(" ")
  print('Threshold: ,' + str(threshold))
  model.load_state_dict(torch.load('/content/hatexplain_roberta_weights_5_GXI_0.8212.pth'))
  evaluate_GXI(model, test_loader, tokenizer,device='cuda', method='threshold', k=5, threshold=threshold)

 
Top K: ,3


Evaluating XAI Metrics: 100%|██████████| 241/241 [01:01<00:00,  3.90it/s]



XAI Metrics:
IoU: 0.0994
Precision: 0.2879
Recall: 0.1552
F1: 0.2017
Faithfulness Drop: 0.0922
 
Top K: ,5


Evaluating XAI Metrics: 100%|██████████| 241/241 [01:01<00:00,  3.90it/s]



XAI Metrics:
IoU: 0.1300
Precision: 0.2884
Recall: 0.2443
F1: 0.2645
Faithfulness Drop: 0.1702
 
Top K: ,10


...


XAI Metrics:
IoU: 0.1371
Precision: 0.2874
Recall: 0.2645
F1: 0.2755
Faithfulness Drop: 0.1733
 
Threshold: ,0.1


Evaluating XAI Metrics: 100%|██████████| 241/241 [01:01<00:00,  3.90it/s]



XAI Metrics:
IoU: 0.0957
Precision: 0.2797
Recall: 0.1624
F1: 0.2055
Faithfulness Drop: 0.1073
 
Threshold: ,0.5


Evaluating XAI Metrics: 100%|██████████| 241/241 [01:01<00:00,  3.91it/s]


XAI Metrics:
IoU: 0.0085
Precision: 0.0251
Recall: 0.0085
F1: 0.0127
Faithfulness Drop: 0.0039


In [47]:
import torch
import torch.nn as nn
from transformers import RobertaModel, RobertaConfig

class CustomRobertaClassifier(nn.Module):
    def __init__(self, pretrained_model_name, hidden_size=128, dropout_rate=0.3):
        super(CustomRobertaClassifier, self).__init__()

        self.roberta = RobertaModel.from_pretrained(pretrained_model_name)
        self.dropout = nn.Dropout(dropout_rate)

        # Intermediate dense layer
        self.dense = nn.Linear(self.roberta.config.hidden_size, hidden_size)
        self.relu = nn.ReLU()

        # Final classification layer (outputting 1 value for binary classification)
        self.classifier = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids = None, attention_mask = None, inputs_embeds = None, output_attentions=False):
      outputs = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions
        )

      pooled_output = outputs.pooler_output  # [batch_size, hidden_size]

      x = self.dropout(pooled_output)
      x = self.relu(self.dense(x))
      x = self.dropout(x)
      logits = self.classifier(x)

      if output_attentions:
        return self.sigmoid(logits), outputs.attentions
      else:
        return self.sigmoid(logits)



tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


model = CustomRobertaClassifier(pretrained_model_name='roberta-base')
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CustomRobertaClassifier(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Lay

In [48]:
import torch

ks = [3,5,10,15,20,25]
thresholds = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
for k in ks:
  print(" ")
  print('Top K: ,' + str(k))
  model.load_state_dict(torch.load('/content/hatexplain_roberta_weights_5_ATTN_0.8331.pth'))
  evaluate_attention_explanations(model, test_loader, tokenizer,device='cuda', method='topk', k=k, threshold=0.01)

for threshold in thresholds:
  print(" ")
  print('Threshold: ,' + str(threshold))
  model.load_state_dict(torch.load('/content/hatexplain_roberta_weights_5_ATTN_0.8331.pth'))
  evaluate_attention_explanations(model, test_loader, tokenizer,device='cuda', method='threshold', k=5, threshold=threshold)

 
Top K: ,3


Evaluating Attention Explanations: 100%|██████████| 241/241 [00:34<00:00,  6.90it/s]



Attention Explanation Metrics:
IoU: 0.3113
Precision: 0.6380
Recall: 0.4655
F1: 0.5383
Faithfulness Drop: 0.1047
 
Top K: ,5


Evaluating Attention Explanations: 100%|██████████| 241/241 [00:34<00:00,  6.93it/s]



Attention Explanation Metrics:
IoU: 0.3309
Precision: 0.5652
Recall: 0.6001
F1: 0.5821
Faithfulness Drop: 0.2090
 
Top K: ,10


...


Attention Explanation Metrics:
IoU: 0.3658
Precision: 0.5877
Recall: 0.6086
F1: 0.5980
Faithfulness Drop: 0.1830
 
Threshold: ,0.1


Evaluating Attention Explanations: 100%|██████████| 241/241 [00:34<00:00,  6.97it/s]



Attention Explanation Metrics:
IoU: 0.3049
Precision: 0.5878
Recall: 0.4313
F1: 0.4976
Faithfulness Drop: 0.0761
 
Threshold: ,0.5


Evaluating Attention Explanations: 100%|██████████| 241/241 [00:34<00:00,  6.98it/s]


Attention Explanation Metrics:
IoU: 0.0729
Precision: 0.1225
Recall: 0.0729
F1: 0.0914
Faithfulness Drop: 0.0036


In [49]:
import torch

ks = [3,5,10,15,20,25]
thresholds = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
for k in ks:
  print(" ")
  print('Top K: ,' + str(k))
  model.load_state_dict(torch.load('/content/hatexplain_roberta_weights_noIG_0.8201.pth'))
  evaluate_attention_explanations(model, test_loader, tokenizer,device='cuda', method='topk', k=k, threshold=0.01)

for threshold in thresholds:
  print(" ")
  print('Threshold: ,' + str(threshold))
  model.load_state_dict(torch.load('/content/hatexplain_roberta_weights_noIG_0.8201.pth'))
  evaluate_attention_explanations(model, test_loader, tokenizer,device='cuda', method='threshold', k=5, threshold=threshold)

 
Top K: ,3


Evaluating Attention Explanations: 100%|██████████| 241/241 [00:34<00:00,  6.96it/s]



Attention Explanation Metrics:
IoU: 0.1152
Precision: 0.3178
Recall: 0.1686
F1: 0.2203
Faithfulness Drop: 0.1743
 
Top K: ,5


Evaluating Attention Explanations: 100%|██████████| 241/241 [00:34<00:00,  6.92it/s]



Attention Explanation Metrics:
IoU: 0.1459
Precision: 0.3078
Recall: 0.2708
F1: 0.2881
Faithfulness Drop: 0.2232
 
Top K: ,10


...


Attention Explanation Metrics:
IoU: 0.1330
Precision: 0.2350
Recall: 0.3332
F1: 0.2756
Faithfulness Drop: 0.2584
 
Threshold: ,0.1


Evaluating Attention Explanations: 100%|██████████| 241/241 [00:34<00:00,  6.98it/s]



Attention Explanation Metrics:
IoU: 0.0651
Precision: 0.1705
Recall: 0.1164
F1: 0.1384
Faithfulness Drop: 0.0840
 
Threshold: ,0.5


Evaluating Attention Explanations: 100%|██████████| 241/241 [00:34<00:00,  7.00it/s]


Attention Explanation Metrics:
IoU: 0.0002
Precision: 0.0020
Recall: 0.0002
F1: 0.0003
Faithfulness Drop: 0.0000


In [16]:
import torch

ks = [3,4,5,6,7,8,9,10]

thresholds = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]

for k in ks:
  print(" ")
  print('Top Features: ,' + str(k))
  model.load_state_dict(torch.load('/content/hatexplain_roberta_weights_LIME_0.8259.pth'))
  evaluate_lime_explanations(model, test_loader, tokenizer,device='cuda', num_features=k, mode='num_features')


for threshold in thresholds:
  print(" ")
  print('Thresh: ,' + str(threshold))
  model.load_state_dict(torch.load('/content/hatexplain_roberta_weights_LIME_0.8259.pth'))
  evaluate_lime_explanations(model, test_loader, tokenizer,device='cuda', num_features=k, mode='threshold', threshold=threshold)


 
Top Features: ,3


Evaluating LIME Explanations: 100%|██████████| 241/241 [09:36<00:00,  2.39s/it]



LIME Explanation Metrics:
IoU: 0.0629
Precision: 0.2413
Recall: 0.0860
F1: 0.1268
Faithfulness Drop: 0.2515
 
Top Features: ,4


Evaluating LIME Explanations: 100%|██████████| 241/241 [09:56<00:00,  2.47s/it]



LIME Explanation Metrics:
IoU: 0.0726
Precision: 0.2523
Recall: 0.1044
F1: 0.1477
Faithfulness Drop: 0.2539
 
Top Features: ,5


...


LIME Explanation Metrics:
IoU: 0.0626
Precision: 0.2461
Recall: 0.0852
F1: 0.1266
Faithfulness Drop: 0.2534
 
Thresh: ,0.1


Evaluating LIME Explanations: 100%|██████████| 241/241 [08:06<00:00,  2.02s/it]



LIME Explanation Metrics:
IoU: 0.0444
Precision: 0.2066
Recall: 0.0558
F1: 0.0878
Faithfulness Drop: 0.2457
 
Thresh: ,0.5


Evaluating LIME Explanations: 100%|██████████| 241/241 [08:05<00:00,  2.02s/it]


LIME Explanation Metrics:
IoU: 0.0086
Precision: 0.0436
Recall: 0.0091
F1: 0.0151
Faithfulness Drop: 0.1434
